In [13]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [14]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [15]:
class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url #store the URL
        response = requests.get(url, headers=HEADERS) #Make a request to the URL
        soup = BeautifulSoup(response.content, 'html.parser') #Parse the HTML
        self.title = soup.title.string if soup.title else "No title found" #Extract the Title
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [16]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [17]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [18]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [23]:
def summarize(url):
    website = Website(url)
    response = requests.post(OLLAMA_API, json={
        "model": MODEL,
        "messages": messages_for(website),
        "stream": False
    }, headers=HEADERS)
    result = response.json()
    return display(Markdown(result["message"]["content"]))

In [24]:
summarize("https://edwarddonner.com")

**Summary of Edward Donner's Website**
=====================================

### About the Author

Edward Donner is a writer, DJ, and entrepreneur who enjoys experimenting with Large Language Models (LLMs). He co-founded Nebula.io, an AI startup that applies AI to help people discover their potential.

### News/Announcements
-----------------------

* **January 23, 2025:** LLM Workshop – Hands-on with Agents – resources available
* **December 21, 2024:** Welcome, SuperDataScientists!
* **November 13, 2024:** Mastering AI and LLM Engineering – Resources
* **October 16, 2024:** From Software Engineer to AI Data Scientist – resources

### Projects

* **Nebula.io**: Applying AI to help people discover their potential and pursue their reason for being.
* **untapt**: An AI startup acquired in 2021.

### Contact Information

* Email: [ed at] edwarddonner [dot] com
* Website: www.edwarddonner.com
* Social Media:
	+ LinkedIn
	+ Twitter
	+ Facebook